In [1]:
"""from google.colab import drive
drive.mount('/content/drive') # file path should be changed"""

In [16]:
import data
import utils
import time
import os

import torch # The top-level PyTorch package and tensor library.
import torch.nn as nn # A subpackage that contains modules and extensible classes for building neural networks.
import torch.optim as optim # A subpackage that contains standard optimization operations like SGD and Adam.
import torch.nn.functional as F # functional interface that contains typical operations used for building neural networks like loss functions and convolutions.

import torchvision # package that provides access to popular datasets, model architectures, and image transformations for computer vision.
import torchvision.transforms as transforms # An interface that contains common transforms for image processing.

torch.set_printoptions(linewidth=120) # to print in one line
torch.set_grad_enabled(True)#to update new gradients have to have past gradients so make pytorch to trach the grad values

from torch.autograd import Variable
from torch.utils.data import DataLoader
"""
import warnings
warnings.filterwarnings('ignore')

from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard"""

"\nimport warnings\nwarnings.filterwarnings('ignore')\n\nfrom torch.utils.tensorboard import SummaryWriter\n%load_ext tensorboard"

In [17]:
"""os.chdir("/content/drive/My Drive/")"""

'os.chdir("/content/drive/My Drive/")'

In [18]:
"""!nvidia-smi # this is to check about the gpu"""

'!nvidia-smi # this is to check about the gpu'

In [19]:
# Setup folders for saved models
if not os.path.exists('saved-models/'):
    os.mkdir('saved-models/')

"""!mkdir saved-models   """

'!mkdir saved-models   '

In [20]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [21]:
# before getting train set have to do several tranformations like making the tensors for the data set
# in randomCrop use one 224 so from both height and width upto 224 is taken
# here several data augmentation concepts are taken place

train_transforms = transforms.Compose([transforms.RandomCrop(224),
                                       utils.RandomRotation(),
                                       utils.RandomTranslation(),
                                       utils.RandomVerticalFlip(),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])

train_set = data.ForestData('data/train_v2.csv', 
                           'data/train-jpg',
                           'data/labels.txt', 
                           train_transforms)

In [22]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.pretrained_model = torchvision.models.resnet18(pretrained=True)
        classifier = [
            nn.Linear(self.pretrained_model.fc.in_features, 17)
        ]
        self.classifier = nn.Sequential(*classifier)
        self.pretrained_model.fc = self.classifier

    def forward(self, x):
        x = self.pretrained_model(x)
        return F.sigmoid(x)


In [23]:
model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
train_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size=100
    ,shuffle=True
)

#criterion = torch.nn.BCELoss()

In [24]:
"""
when using crossentropy loss shouldn't one-hot encode target variable y. 
Pytorch crossentropy expects just the class indices as target not their one-hot encoded version.
"""
#tb = SummaryWriter()
for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 
        labels = labels.argmax(dim=1) # convert one hot encoding to label
        
        preds = model(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss
        
        optimizer.zero_grad()# always make to zero as in every steps has to get the new gradients so already existing value is made to zero
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )
    
    """tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)"""

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 

In [ ]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html -----  study this

PATH = "saved-models/entire_model_resnet18.pt"    
torch.save(model.state_dict(), PATH)    

In [ ]:
PATH = "saved-models/entire_model_ex_resnet18.pt"   
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, PATH)

In [ ]:
"""tensorboard --logdir=runs"""